<a href="https://colab.research.google.com/github/alepenaa94/TP1_Real_or_Not/blob/master/TP1_Real_or_Not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamiento de datos

En este notebook la idea es realizar todo el procesamiento del set de datos, limpieza en los campos de ser necesario y sección para la generacion de los features.

## Ideas de cosas a realizar

--> Generar un N-grama <br>
--> mejorar los stopwords <br>
--> poder darle mas caracter a un tweet segun su texto <br>
--> TF-IDF? dijeron que podía ser útil, habría que ver de que manera lo implementamos <br>

In [1]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('./Data/train.csv', encoding='latin-1',dtype={'id': np.uint16,'target': np.bool})
test_df = pd.read_csv('./Data/test.csv', encoding='latin-1',dtype={'id': np.uint16})

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,True
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,True
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,True
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",True
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,True


In [4]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Limpieza de la información

En base a lo aprendido en el TP1, limpiamos la información de algunos campos del dataframe.

In [5]:
train_df['keyword'] = train_df['keyword'].str.replace('%20',' ')
test_df['keyword'] = test_df['keyword'].str.replace('%20',' ')

In [6]:
train_df['location'].fillna('Unknown',inplace=True)
test_df['location'].fillna('Unknown',inplace=True)

## Creación de features

Volvemos a generar las columnas utilizadas para el TP1 ya que podrían resultar features útiles para el modelo de predicción.

###### Cantidad de palabras en el tweet

In [7]:
train_df['cantidad_de_palabras'] = train_df['text'].str.count(' ') + 1
test_df['cantidad_de_palabras'] = test_df['text'].str.count(' ') + 1

###### Longitud del tweet

In [8]:
train_df['longitud_del_tweet'] = train_df['text'].str.len()
test_df['longitud_del_tweet'] = test_df['text'].str.len()

###### Realiza menciones

In [9]:
train_df['tiene_menciones'] = train_df['text'].str.contains('@')
test_df['tiene_menciones'] = test_df['text'].str.contains('@')

###### Tweet expresivo

In [10]:
train_df['es_expresivo'] = (train_df['text'].str.contains('\!\!') | train_df['text'].str.contains('\?\?'))
test_df['es_expresivo'] = (test_df['text'].str.contains('\!\!') | test_df['text'].str.contains('\?\?'))

###### Cantidad de hashtags

In [11]:
train_df['cantidad_de_hashtags'] = train_df['text'].str.count('#')
test_df['cantidad_de_hashtags'] = test_df['text'].str.count('#')

###### Tiene limks

In [12]:
train_df['tiene_links'] = train_df['text'].str.contains('http')
test_df['tiene_links'] = test_df['text'].str.contains('http')

In [25]:
# Guardo en otros archivos los DF train y test modificados para usarlos en diferentes algoritmos
train_df.to_csv('train_new.csv')
test_df.to_csv('test_new.csv')

## Prueba Random Forest

Pruebo para ver el formato para los submit usando unos pocos features, despues lo limpiamos y lo separamos en otro notebook con más algoritmos.

In [13]:
X = train_df.sort_values(by='id',ascending=True).iloc[:,5:].values
Y = train_df.sort_values(by='id',ascending=True).iloc[:,4].values

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [15]:
#from sklearn.preprocessing import StandardScaler

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

accuracy = 0
cant_arboles_optimos = 0

for i in range(100):
    classifier = RandomForestClassifier(n_estimators=i+1, random_state=1)
    classifier.fit(X_train, Y_train)
    y_pred = classifier.predict(X_test)
        
    if (accuracy < accuracy_score(Y_test,y_pred)):
        accuracy = accuracy_score(Y_test,y_pred)
        cant_arboles_optimos = i+1

In [17]:
print('Arboles:',cant_arboles_optimos)
print('Accuracy:',accuracy)

Arboles: 68
Accuracy: 0.6631648063033486


In [18]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=cant_arboles_optimos, random_state=1)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(Y_test,y_pred))
print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test, y_pred))

[[640 242]
 [271 370]]
              precision    recall  f1-score   support

       False       0.70      0.73      0.71       882
        True       0.60      0.58      0.59       641

    accuracy                           0.66      1523
   macro avg       0.65      0.65      0.65      1523
weighted avg       0.66      0.66      0.66      1523

0.6631648063033486


## Prueba generar submit

Pruebo hacer un submit con esta predicción inicial, después movemos esta parte a otro notebook.

In [20]:
registros_a_predecir = test_df.sort_values(by='id',ascending=True).iloc[:,4:].values

In [21]:
prediccion = classifier.predict(registros_a_predecir)

Damos el formato para realizar el submit.

In [22]:
resultado = pd.DataFrame(pd.Series(prediccion))
resultado.insert(loc=0, column='id', value=(test_df.sort_values(by='id',ascending=True)['id']))
resultado.rename(columns={0:'target'},inplace=True)
resultado.set_index('id',inplace=True)
resultado['target'] = resultado['target'].astype('uint8')
resultado

,target
id,
0,0
2,1
3,0
9,0
11,0
12,0
21,0
22,0
27,0


In [23]:
resultado.to_csv('prediccion_RF_inicial.csv')